# AEM Reference model constraint

### NOTE: this notebook is not complete and still requires more work. The constrained conductivity and weighting values are not constrained laterally.

In [19]:
#Import libraries
import pandas as pd
import numpy as np
import xarray as xr

# User requirement: enter details of input datasets

In [21]:
##User requirement: enter the following details for input datasets:

##Dataset location. Ensure "/" are used, not "\"
folder= str(r"C:/Users/u67397/AnacondaProjects/aem/input_data/")

##Name of the datafile. Include the ".extension"
dat_name= str("AA140005_Line1080.dat")

##Name of the Control file. Include the ".extension"
con_name= str("galeisbstdem.con")

##Name of constraint file. Include the ".extension"
constraint_name=str("input_test.csv") 

# User requirement: enter the column location for certain data values

In [22]:
##User requirement: enter the column location within the datafile.
##please do **not** use programmer counting, for example the third column should be "3"

line_column = 1 ##The column location of the "Line number"
easting_column = 3 ##The column location of "easting/x/longitude" values
northing_column = 4 ##The column location of "northing/y/latitude" values

# User requirement: enter reference model data variables

In [23]:
##User requirement: enter the following values relating to the reference model

##Enter the reference model conductivity 
reference_conductivity= 0.001

##Enter the number of layers
reference_layers= 30

##Set constraint weighting. 1% =no constraint (e.g. reference model) and 10%= hard constraint at location
no_constraint=1 #Value given to represent reference model
hard_constraint=10 #Value given to represent constraint data
lateral_constraint=3 #The lateral constraint distance (number of soundings each side of the constraint location)

# User requirement: enter folder location to save files

In [24]:
##User requirement: enter the output folder location. Ensure "/" are used, not "\"
output_folder= str(r"C:/Users/u67397/AnacondaProjects/aem/output_data/")

# No more user requirements... Just run the remaining cells

In [6]:
#Location variables are defined. Because python locates columns starting at 0, rather than 1, column values are subtracted 
easting_column2= easting_column - 1 #Set the location of the easting column within input datafile
northing_column2= northing_column - 1 #Set the location of the northing column within input datafile
line_column2= line_column -1 #Set the location of the line title column within input datafile

In [7]:
#Data is imported
dat= pd.read_fwf(folder+dat_name, header=None) #Import data file
dat2=dat.set_index([easting_column2,northing_column2], drop=False) #Set coordinates as index
dat2['all_sumcoord']=dat2[easting_column2]+dat2[northing_column2] #Create a new column that combines coordinates
datlen= len(dat2.index) #Create variable of dat length

In [33]:
#Thickness data from Control file is imported and formatted
con = pd.read_csv (folder+con_name, header=None) #Import control file
con.replace(regex=True,to_replace=r'\t', value=r'', inplace=True)#Remove tabs
tlayers=con.loc[con[0].str.contains('Thickness      = ')] #Extract thickness values saved in the Control file and save in dataframe
thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')#Remove the words"Thickness = " from dataframe
thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan) #Split values and replace spaces with "nan"
thicknessoflayers.dropna(axis=1,inplace=True) #Remove nan values
thicknessoflayers=thicknessoflayers.reset_index(drop=True).T.reset_index(drop=True).astype(float) #Reset index

#The bottom layer is duplicated
bottomlayer=thicknessoflayers[0].iloc[-1] #Save bottom layer as variable
bottomlayer=np.array([bottomlayer]) #Set value as array
bottomlayer=pd.DataFrame(bottomlayer) #Set value as dataframe
thicknessoflayers=thicknessoflayers.append(bottomlayer).reset_index(drop=True) #Append bottom layer to thickness dataframe
depthoflayers=np.cumsum(thicknessoflayers).T #Create a cumulative sum of thickness values
depthoflayers=(pd.np.tile(depthoflayers, (datlen, 1))) #Create an array of depth values (soundings x layers)
depthoflayers=pd.DataFrame(depthoflayers) #Convert to dataframe

#The top layer is defined as a variable
all_layers= depthoflayers[0:1] #Save top depths as dataframe 
all_layers=np.array(all_layers).squeeze() #Convert from dataframe to array and squeeze into correct format

In [9]:
#An array with the number of reference model layers is created
numberoflocations=list(range(reference_layers)) #Create a list of the number of reference model layers
numberoflocations=np.asarray(numberoflocations) #Turn list into an array

In [10]:
#A dataframe of reference reference conductivity values is created
RC=pd.Series(reference_conductivity) #Turn reference conductivity into a series
RC=(pd.np.tile(RC, (datlen, reference_layers))) #Create an array of reference conductivity values (soundings x layers)
RC=pd.DataFrame(RC) #Convert array into dataframe

In [11]:
#Constraint file is loaded and processed
constraint_data= pd.read_csv(folder+constraint_name) #Import control file
constraint_data['constraint_coords_sum']=constraint_data['easting']+constraint_data['northing'] #Take sum of easting and northing
constraint_data=constraint_data.set_index(['ID'], drop=False)#Set ID as index
constraint_data #Print dataframe to screen

,ID,easting,northing,layer_top,layer_bottom,layer_conductivity,constraint_coords_sum
ID,,,,,,,
erin,erin,220955.6,6867443.6,0,10,0.05,7088399.2
rules,rules,220956.6,6867444.6,10,20,0.10,7088401.2


In [12]:
#Creation of a data array made up of constraint weighting values
constraint_weighting=np.full((datlen,reference_layers),no_constraint) #Create an array of constraint weighting values (soundings x layers)

In [30]:
#Coordinates are extracted and processed
easting_coords=pd.DataFrame(dat2.iloc[:,(easting_column -1):(easting_column)]) #Extract easting values from data file
easting_coords=easting_coords.transpose() #Transpose dataframe
easting_coords=np.array(easting_coords).squeeze() #Convert from dataframe to array and squeeze into correct format

northing_coords=pd.DataFrame(dat2.iloc[:,(northing_column -1):(northing_column)]) #Extract northing values from data file
northing_coords=northing_coords.transpose() #Transpose dataframe
northing_coords=np.array(northing_coords).squeeze()  #Convert from dataframe to array and squeeze into correct format

northing_array=pd.np.tile(northing_coords, ((len(numberoflocations), 1))).transpose() #Create an array of northing values (soundings x layers)

coords_index=pd.DataFrame(dat.iloc[:,(easting_column2):(northing_column2 +1)]).set_index(easting_column-1) #Set coordinates for use in index

In [14]:
#Coordinates within the constraint data are matched with the closest coordinates in the datafile
matched=pd.DataFrame([0,0,0]).transpose().set_index([0]) #Create empty dataframe

for all in range(len(constraint_data)): #Match coordinates through loop
    constraint_ID=(constraint_data.ID[all])
    constraint_coords=(constraint_data.constraint_coords_sum[all]) 
    dat2[str(constraint_ID)]= np.abs(dat2.all_sumcoord-constraint_coords)  
    matched1=np.argmin(dat2[str(constraint_ID)]) 
    e_matched=matched1[0]
    n_matched=matched1[1]
    matched2=pd.DataFrame([constraint_ID,float(e_matched),float(n_matched)]).transpose().set_index([0]).astype(float)
    matched=pd.concat([matched,matched2], axis=0)

matched=matched.rename(columns={1:'east_matched',2:'north_matched'}).drop([0]) #Rename coordinate headings

In [34]:
#Matched coordinates are matched to the constraint dataframe
constraint_data2= pd.concat([constraint_data, matched],axis=1) #Join matched coordinates and constraint data
constraint_data2=constraint_data2.set_index(['east_matched','north_matched'],drop=False) #Set matched coordinates as index
constraint_data2 #Print dataframe to screen

,,ID,easting,northing,layer_top,layer_bottom,layer_conductivity,constraint_coords_sum,east_matched,north_matched
east_matched,north_matched,,,,,,,,,
220955.6,6867443.6,erin,220955.6,6867443.6,0,10,0.05,7088399.2,220955.6,6867443.6
220958.6,6867443.7,rules,220956.6,6867444.6,10,20,0.10,7088401.2,220958.6,6867443.7


In [16]:
#A multidimensional array made up of coordinates, reference conductivity, 
#layer thicknesses, and weighting is crated.
#At the location of constraint, the conductivity and weighting values have been replaced

for y in range(len(constraint_data2)): #Create loop and define values
    EC_value1=constraint_data2['layer_conductivity'][y]
    z_top=constraint_data2['layer_top'][y]
    z_bottom=constraint_data2['layer_bottom'][y]
    east=float("{0:.2f}".format(constraint_data2['east_matched'][y]))
    north=float("{0:.2f}".format(constraint_data2['north_matched'][y]))    
    
    #Convert datasets into arrays 
    coords_da1={'easting': easting_coords, 'depth':all_layers} #Create EC array
    rc_da = xr.DataArray(RC, dims=('easting', 'depth'), coords=coords_da1)
    
    coords_da2={'easting': easting_coords, 'layers': numberoflocations}#Create thickness array
    thickness_da = xr.DataArray(depthoflayers, dims=('easting', 'layers'), coords=coords_da2)
    
    coords_da3={'easting': easting_coords, 'depth': all_layers}
    northing_da = xr.DataArray(northing_array, dims=('easting','depth'), coords=coords_da3)
    
    coords_da4={'easting': easting_coords, 'depth': all_layers}
    weighting_da = xr.DataArray(constraint_weighting, dims=('easting','depth'), coords=coords_da4)
    
    dataset = xr.Dataset({'rc_dataset': rc_da, 'thickness_dataset': thickness_da, 'northing_dataset':northing_da, 'weighting_dataset':weighting_da}) #Create combined EC and thickness array
    
    #Change EC and weighting values at constrained locations/depths
    array2=dataset.assign(ec=EC_value1).where((thickness_da>z_top)&(thickness_da<z_bottom)).sel(easting=east)
    array2['ec']=array2.ec.fillna(reference_conductivity)
    array3=np.array(array2['ec'])
    array4=dataset.assign(weighting=hard_constraint).where((thickness_da>z_top)&(thickness_da<z_bottom)).sel(easting=east) 
    print()
    array4['weighting']=array4.weighting.fillna(no_constraint)
    array5=np.array(array4['weighting'])
    
    #Create array with updated EC and weighting values at certain depths
    rc_da.loc[east]=array3
    weighting_da.loc[east]=array5
    dataset = xr.Dataset({'rc_dataset': rc_da, 'thickness_dataset': thickness_da, 'northing_dataset':northing_da, 'weighting_dataset':weighting_da}) #Create combined EC and thickness array

In [17]:
#Concatenate original datafile with the new reference conductivity array and the new weighting array
rc_df=rc_da.to_pandas() #Convert to from array to dataframe
weighting_df=weighting_da.to_pandas() #Convert to from array to dataframe
rc_weighting=pd.concat([rc_df,weighting_df],axis=1) #Join conductivity and weighting dataframes
rc_weighting['northing']=coords_index #Add northing values
rc_weighting=rc_weighting.set_index('northing',append=True) #Set index
dat_final=pd.concat([dat2,rc_weighting],axis=1) #Join original datafile with updated constrained data
dat_final=dat_final.reset_index(drop=True).T.reset_index(drop=True).T #Reset indexes

In [18]:
#The new datafile is saved as .csv and as .dat
dat_final.to_csv(output_folder+(dat_name[:-4])+'_final.dat')
dat_final.to_csv(output_folder+(dat_name[:-4])+'_final.csv')

# The following code relates to laterally constraining conductivity and weighting values. The code below is only a concept.

In [51]:
#Concept for laterally constraining values

lateral_test_dataset=np.zeros((15,11)) #Create test dataset
lateral_test_dataset[:2,5]=hard_constraint  #Add hard constraint

#Return evenly spaced numbers at the specified number of soundings (lateral_constraint)
lateral_test_max=np.linspace(hard_constraint,no_constraint,num=lateral_constraint,endpoint=False)
lateral_test_min=np.linspace(no_constraint,hard_constraint,num=lateral_constraint,endpoint=False)

for row in lateral_test_dataset: #Create a loop that finds any values above 0 (e.g. hard constraint values)
    if row.any()>0:
        ixx = np.where(row==hard_constraint) #set variable where values = hard constraint
        ixx= int(ixx[0]) #set variable for location of hard constraint
        
        ixx_min_down=ixx-lateral_constraint #Set variable for minimum lateral constraint
        ixx_min_up=ixx #Set variable for minimum lateral constraint
            
        ixx_max_down=ixx #Set variable for maximum lateral constraint
        ixx_max_up=ixx+lateral_constraint  #Set variable for maximum lateral constraint
        
        row[ixx_min_down:ixx_min_up]=lateral_test_min #In locations set above, replace with lateral constrained values
        row[ixx_max_down:ixx_max_up]=lateral_test_max #In locations set above, replace with lateral constrained values


np.place(lateral_test_dataset,lateral_test_dataset==0,1) #Replace 0 with 1

print (lateral_test_dataset) #Print result to screen


[[  1.   1.   1.   4.   7.  10.   7.   4.   1.   1.   1.]
 [  1.   1.   1.   4.   7.  10.   7.   4.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]]
